In [1]:
import pandas as pd
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
from torch.utils.data import DataLoader
import torchvision
import torch.utils.data as data_utils
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import matplotlib.pyplot as plt
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
import glob

l = [pd.read_pickle(filename) for filename in glob.glob("./erml_data/*.pkl")]

raw_df = pd.concat(l, axis=0,ignore_index=True).sample(frac=1)
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16018500 entries, 8082717 to 4429192
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Temperature  float64
 1   GSR          float64
 2   EOG1         float64
 3   EOG2         float64
 4   EEG1         float64
 5   EEG2         float64
 6   RED_RAW      float64
 7   IR_RAW       float64
 8   Arousal      int64  
 9   Dominance    int64  
 10  Valence      int64  
dtypes: float64(8), int64(3)
memory usage: 1.4 GB


In [3]:
msk = np.random.rand(len(raw_df)) < 0.8

train = raw_df[msk]

test = raw_df[~msk]
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12815887 entries, 8082717 to 4429192
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Temperature  float64
 1   GSR          float64
 2   EOG1         float64
 3   EOG2         float64
 4   EEG1         float64
 5   EEG2         float64
 6   RED_RAW      float64
 7   IR_RAW       float64
 8   Arousal      int64  
 9   Dominance    int64  
 10  Valence      int64  
dtypes: float64(8), int64(3)
memory usage: 1.1 GB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3202613 entries, 1591730 to 15251090
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Temperature  float64
 1   GSR          float64
 2   EOG1         float64
 3   EOG2         float64
 4   EEG1         float64
 5   EEG2         float64
 6   RED_RAW      float64
 7   IR_RAW       float64
 8   Arousal      int64  
 9   Dominance    int64  
 10  Valence      int64  
dtypes: float64(8), int64(3)
memor

In [4]:
net = torch.nn.Sequential(
        torch.nn.Linear(8, 200),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(200, 100),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(100, 3),
    )

In [5]:
train

,Temperature,GSR,EOG1,EOG2,EEG1,EEG2,RED_RAW,IR_RAW,Arousal,Dominance,Valence
8082717,15317.0,15761480.0,16753359.0,16753322.0,12049399.0,12922664.0,44051.0,73262.0,6,7,7
4980229,15373.0,16774654.0,16642274.0,16642271.0,15408504.0,2856392.0,41482.0,71251.0,9,8,7
4139565,15395.0,15598382.0,290085.0,289978.0,13427195.0,9671076.0,69493.0,86962.0,2,4,5
3399202,15464.0,1033942.0,523288.0,16776180.0,13083872.0,9289845.0,72889.0,93895.0,8,5,7
14481573,15282.0,1617404.0,294336.0,294258.0,13799077.0,14140719.0,63771.0,87780.0,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...
1308427,15425.0,16504025.0,251166.0,251113.0,7810886.0,876458.0,65566.0,89586.0,5,4,4
1828062,15316.0,15150103.0,203351.0,203288.0,15951752.0,1134123.0,70416.0,89587.0,5,9,7
14795434,15333.0,174385.0,311126.0,311010.0,8974360.0,14468862.0,1412.0,1821.0,5,4,5
5089202,15435.0,146749.0,16644565.0,16644556.0,63961.0,1110288.0,36908.0,67778.0,9,8,7


In [6]:
normalized_train_df=(train-train.min())/(train.max()-train.min())
normalized_test_df=(test-test.min())/(test.max()-test.min())

In [20]:
normalized_train_df

,Temperature,GSR,EOG1,EOG2,EEG1,EEG2,RED_RAW,IR_RAW,Arousal,Dominance,Valence
8082717,0.177794,0.939457,0.998578,0.998576,0.718200,0.770251,0.486858,0.675873,0.625,0.750,0.750
4980229,0.178725,0.999847,0.991957,0.991957,0.918418,0.170254,0.458379,0.657216,1.000,0.875,0.750
4139565,0.179090,0.929736,0.017290,0.017284,0.800323,0.576441,0.768898,0.802974,0.125,0.375,0.500
3399202,0.180237,0.061628,0.031190,0.999938,0.779860,0.553718,0.806545,0.867295,0.875,0.500,0.750
14481573,0.177212,0.096405,0.017544,0.017539,0.822489,0.842853,0.705466,0.810563,0.125,0.125,0.125
...,...,...,...,...,...,...,...,...,...,...,...
1308427,0.179589,0.983717,0.014971,0.014968,0.465565,0.052241,0.725365,0.827318,0.500,0.375,0.375
1828062,0.177777,0.903017,0.012121,0.012117,0.950799,0.067599,0.779130,0.827328,0.500,1.000,0.750
14795434,0.178060,0.010394,0.018545,0.018538,0.534914,0.862411,0.014179,0.013081,0.500,0.375,0.500
5089202,0.179755,0.008747,0.992093,0.992093,0.003812,0.066178,0.407673,0.624995,1.000,0.875,0.750


In [7]:
X_train = normalized_train_df.iloc[:,:-3]
y_train = normalized_train_df.iloc[:,-3:]
y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12815887 entries, 8082717 to 4429192
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Arousal    float64
 1   Dominance  float64
 2   Valence    float64
dtypes: float64(3)
memory usage: 391.1 MB


In [8]:
X_test = normalized_test_df.iloc[:,:-3]
y_test = normalized_test_df.iloc[:,-3:]
y_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3202613 entries, 1591730 to 15251090
Data columns (total 3 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Arousal    float64
 1   Dominance  float64
 2   Valence    float64
dtypes: float64(3)
memory usage: 97.7 MB


In [9]:
target = torch.tensor(y_train.values)
features = torch.tensor(X_train.values)

train = data_utils.TensorDataset(features, target)
train_loader = data_utils.DataLoader(train, batch_size=400, shuffle=True)

In [10]:
target_test = torch.tensor(y_test.values)
features_test = torch.tensor(X_test.values)

test = data_utils.TensorDataset(features_test, target_test)
test_loader = data_utils.DataLoader(test, batch_size=250, shuffle=True)


In [11]:
criterion= nn.MSELoss() #L1Loss
optimizer = optim.Adam(net.parameters(),lr =1e-3)
epochs = 1 #2
for epoch in range(epochs):
    for batch_idx,(data,target) in enumerate (train_loader):
        
        optimizer.zero_grad()
        output = net (data.float())
        loss = criterion(output.float(),target.float())
        loss.backward()
        optimizer.step()
        
        print('Epoch {} batch idx {}, loss {}'.format(epoch,batch_idx,loss.item()))

Epoch 0 batch idx 0, loss 0.37177425622940063
Epoch 0 batch idx 1, loss 0.29636189341545105
Epoch 0 batch idx 2, loss 0.2450752705335617
Epoch 0 batch idx 3, loss 0.19918029010295868
Epoch 0 batch idx 4, loss 0.1465727537870407
Epoch 0 batch idx 5, loss 0.1331649124622345
Epoch 0 batch idx 6, loss 0.09962690621614456
Epoch 0 batch idx 7, loss 0.09488137066364288
Epoch 0 batch idx 8, loss 0.08066655695438385
Epoch 0 batch idx 9, loss 0.08366843312978745
Epoch 0 batch idx 10, loss 0.08744089305400848
Epoch 0 batch idx 11, loss 0.09073865413665771
Epoch 0 batch idx 12, loss 0.09678906202316284
Epoch 0 batch idx 13, loss 0.0903429463505745
Epoch 0 batch idx 14, loss 0.08997447043657303
Epoch 0 batch idx 15, loss 0.08808965235948563
Epoch 0 batch idx 16, loss 0.08739884942770004
Epoch 0 batch idx 17, loss 0.07994259148836136
Epoch 0 batch idx 18, loss 0.08072525262832642
Epoch 0 batch idx 19, loss 0.07088411599397659
Epoch 0 batch idx 20, loss 0.07692557573318481
Epoch 0 batch idx 21, loss 

Epoch 0 batch idx 181, loss 0.055957697331905365
Epoch 0 batch idx 182, loss 0.056682389229536057
Epoch 0 batch idx 183, loss 0.05351165682077408
Epoch 0 batch idx 184, loss 0.055575281381607056
Epoch 0 batch idx 185, loss 0.05407072603702545
Epoch 0 batch idx 186, loss 0.052597686648368835
Epoch 0 batch idx 187, loss 0.05455131456255913
Epoch 0 batch idx 188, loss 0.05624566972255707
Epoch 0 batch idx 189, loss 0.054324205964803696
Epoch 0 batch idx 190, loss 0.0584435798227787
Epoch 0 batch idx 191, loss 0.058115195482969284
Epoch 0 batch idx 192, loss 0.059415239840745926
Epoch 0 batch idx 193, loss 0.05099111795425415
Epoch 0 batch idx 194, loss 0.05115912854671478
Epoch 0 batch idx 195, loss 0.054652098566293716
Epoch 0 batch idx 196, loss 0.05215203016996384
Epoch 0 batch idx 197, loss 0.05814726650714874
Epoch 0 batch idx 198, loss 0.052663933485746384
Epoch 0 batch idx 199, loss 0.05680117756128311
Epoch 0 batch idx 200, loss 0.05617614835500717
Epoch 0 batch idx 201, loss 0.05

Epoch 0 batch idx 353, loss 0.046897318214178085
Epoch 0 batch idx 354, loss 0.05504963546991348
Epoch 0 batch idx 355, loss 0.04901580512523651
Epoch 0 batch idx 356, loss 0.052853528410196304
Epoch 0 batch idx 357, loss 0.05321355536580086
Epoch 0 batch idx 358, loss 0.046440206468105316
Epoch 0 batch idx 359, loss 0.04588022455573082
Epoch 0 batch idx 360, loss 0.04740661755204201
Epoch 0 batch idx 361, loss 0.04923110455274582
Epoch 0 batch idx 362, loss 0.04963348060846329
Epoch 0 batch idx 363, loss 0.04952261224389076
Epoch 0 batch idx 364, loss 0.046675167977809906
Epoch 0 batch idx 365, loss 0.04784851893782616
Epoch 0 batch idx 366, loss 0.05073948949575424
Epoch 0 batch idx 367, loss 0.050070472061634064
Epoch 0 batch idx 368, loss 0.04839060828089714
Epoch 0 batch idx 369, loss 0.051462322473526
Epoch 0 batch idx 370, loss 0.049915824085474014
Epoch 0 batch idx 371, loss 0.04694877937436104
Epoch 0 batch idx 372, loss 0.04873953014612198
Epoch 0 batch idx 373, loss 0.051399

Epoch 0 batch idx 526, loss 0.0434969998896122
Epoch 0 batch idx 527, loss 0.044278040528297424
Epoch 0 batch idx 528, loss 0.04875621199607849
Epoch 0 batch idx 529, loss 0.045796867460012436
Epoch 0 batch idx 530, loss 0.045288119465112686
Epoch 0 batch idx 531, loss 0.04908040165901184
Epoch 0 batch idx 532, loss 0.045291025191545486
Epoch 0 batch idx 533, loss 0.04553253948688507
Epoch 0 batch idx 534, loss 0.04325299710035324
Epoch 0 batch idx 535, loss 0.04783684015274048
Epoch 0 batch idx 536, loss 0.04094037413597107
Epoch 0 batch idx 537, loss 0.04502807930111885
Epoch 0 batch idx 538, loss 0.04661962017416954
Epoch 0 batch idx 539, loss 0.04442587122321129
Epoch 0 batch idx 540, loss 0.043386295437812805
Epoch 0 batch idx 541, loss 0.04098650440573692
Epoch 0 batch idx 542, loss 0.04345184937119484
Epoch 0 batch idx 543, loss 0.041624970734119415
Epoch 0 batch idx 544, loss 0.04091580957174301
Epoch 0 batch idx 545, loss 0.041647762060165405
Epoch 0 batch idx 546, loss 0.0446

Epoch 0 batch idx 710, loss 0.0389564149081707
Epoch 0 batch idx 711, loss 0.038861233741045
Epoch 0 batch idx 712, loss 0.03517217934131622
Epoch 0 batch idx 713, loss 0.03592461720108986
Epoch 0 batch idx 714, loss 0.039216894656419754
Epoch 0 batch idx 715, loss 0.037344399839639664
Epoch 0 batch idx 716, loss 0.04111369699239731
Epoch 0 batch idx 717, loss 0.03920198604464531
Epoch 0 batch idx 718, loss 0.038632117211818695
Epoch 0 batch idx 719, loss 0.04264945164322853
Epoch 0 batch idx 720, loss 0.039367347955703735
Epoch 0 batch idx 721, loss 0.04023529961705208
Epoch 0 batch idx 722, loss 0.03707057982683182
Epoch 0 batch idx 723, loss 0.041041985154151917
Epoch 0 batch idx 724, loss 0.037234097719192505
Epoch 0 batch idx 725, loss 0.04183673486113548
Epoch 0 batch idx 726, loss 0.03495008870959282
Epoch 0 batch idx 727, loss 0.038744255900382996
Epoch 0 batch idx 728, loss 0.041013654321432114
Epoch 0 batch idx 729, loss 0.04277916997671127
Epoch 0 batch idx 730, loss 0.03736

...

Epoch 0 batch idx 31296, loss 0.019317448139190674
Epoch 0 batch idx 31297, loss 0.019113855436444283
Epoch 0 batch idx 31298, loss 0.019270259886980057
Epoch 0 batch idx 31299, loss 0.01784096471965313
Epoch 0 batch idx 31300, loss 0.017638998106122017
Epoch 0 batch idx 31301, loss 0.02157842181622982
Epoch 0 batch idx 31302, loss 0.019366562366485596
Epoch 0 batch idx 31303, loss 0.020429737865924835
Epoch 0 batch idx 31304, loss 0.01919223554432392
Epoch 0 batch idx 31305, loss 0.023719273507595062
Epoch 0 batch idx 31306, loss 0.020721174776554108
Epoch 0 batch idx 31307, loss 0.017738426104187965
Epoch 0 batch idx 31308, loss 0.021273065358400345
Epoch 0 batch idx 31309, loss 0.018818510696291924
Epoch 0 batch idx 31310, loss 0.018255671486258507
Epoch 0 batch idx 31311, loss 0.02065354399383068
Epoch 0 batch idx 31312, loss 0.01747182570397854
Epoch 0 batch idx 31313, loss 0.018562786281108856
Epoch 0 batch idx 31314, loss 0.019574346020817757
Epoch 0 batch idx 31315, loss 0.0200

Epoch 0 batch idx 31458, loss 0.01796511560678482
Epoch 0 batch idx 31459, loss 0.02379872091114521
Epoch 0 batch idx 31460, loss 0.023074164986610413
Epoch 0 batch idx 31461, loss 0.019043240696191788
Epoch 0 batch idx 31462, loss 0.01961704157292843
Epoch 0 batch idx 31463, loss 0.020767606794834137
Epoch 0 batch idx 31464, loss 0.01956718973815441
Epoch 0 batch idx 31465, loss 0.021473554894328117
Epoch 0 batch idx 31466, loss 0.01963041163980961
Epoch 0 batch idx 31467, loss 0.02107865735888481
Epoch 0 batch idx 31468, loss 0.01913035474717617
Epoch 0 batch idx 31469, loss 0.021464692428708076
Epoch 0 batch idx 31470, loss 0.019655028358101845
Epoch 0 batch idx 31471, loss 0.022347528487443924
Epoch 0 batch idx 31472, loss 0.02255825698375702
Epoch 0 batch idx 31473, loss 0.023930296301841736
Epoch 0 batch idx 31474, loss 0.020923804491758347
Epoch 0 batch idx 31475, loss 0.022155338898301125
Epoch 0 batch idx 31476, loss 0.020097097381949425
Epoch 0 batch idx 31477, loss 0.0192606

Epoch 0 batch idx 31620, loss 0.019750254228711128
Epoch 0 batch idx 31621, loss 0.020935073494911194
Epoch 0 batch idx 31622, loss 0.02026154287159443
Epoch 0 batch idx 31623, loss 0.021752482280135155
Epoch 0 batch idx 31624, loss 0.02217794954776764
Epoch 0 batch idx 31625, loss 0.019276324659585953
Epoch 0 batch idx 31626, loss 0.017083466053009033
Epoch 0 batch idx 31627, loss 0.020245563238859177
Epoch 0 batch idx 31628, loss 0.02003484033048153
Epoch 0 batch idx 31629, loss 0.01984044350683689
Epoch 0 batch idx 31630, loss 0.01999487169086933
Epoch 0 batch idx 31631, loss 0.018259787932038307
Epoch 0 batch idx 31632, loss 0.019012324512004852
Epoch 0 batch idx 31633, loss 0.02031373605132103
Epoch 0 batch idx 31634, loss 0.017121752724051476
Epoch 0 batch idx 31635, loss 0.01860908791422844
Epoch 0 batch idx 31636, loss 0.019195014610886574
Epoch 0 batch idx 31637, loss 0.018088771030306816
Epoch 0 batch idx 31638, loss 0.017678888514637947
Epoch 0 batch idx 31639, loss 0.018274

Epoch 0 batch idx 31791, loss 0.017797023057937622
Epoch 0 batch idx 31792, loss 0.020907819271087646
Epoch 0 batch idx 31793, loss 0.021091453731060028
Epoch 0 batch idx 31794, loss 0.02041095495223999
Epoch 0 batch idx 31795, loss 0.0221615731716156
Epoch 0 batch idx 31796, loss 0.017281578853726387
Epoch 0 batch idx 31797, loss 0.020580478012561798
Epoch 0 batch idx 31798, loss 0.018155600875616074
Epoch 0 batch idx 31799, loss 0.021175971254706383
Epoch 0 batch idx 31800, loss 0.018044667318463326
Epoch 0 batch idx 31801, loss 0.01875055395066738
Epoch 0 batch idx 31802, loss 0.0193049106746912
Epoch 0 batch idx 31803, loss 0.0190606489777565
Epoch 0 batch idx 31804, loss 0.02299070730805397
Epoch 0 batch idx 31805, loss 0.017171410843729973
Epoch 0 batch idx 31806, loss 0.018052948638796806
Epoch 0 batch idx 31807, loss 0.02073650248348713
Epoch 0 batch idx 31808, loss 0.021076401695609093
Epoch 0 batch idx 31809, loss 0.0181750375777483
Epoch 0 batch idx 31810, loss 0.01901943422

Epoch 0 batch idx 31960, loss 0.019823923707008362
Epoch 0 batch idx 31961, loss 0.02057202160358429
Epoch 0 batch idx 31962, loss 0.02206532284617424
Epoch 0 batch idx 31963, loss 0.02014678716659546
Epoch 0 batch idx 31964, loss 0.01926816813647747
Epoch 0 batch idx 31965, loss 0.022068405523896217
Epoch 0 batch idx 31966, loss 0.016638323664665222
Epoch 0 batch idx 31967, loss 0.019875528290867805
Epoch 0 batch idx 31968, loss 0.02022658661007881
Epoch 0 batch idx 31969, loss 0.022075895220041275
Epoch 0 batch idx 31970, loss 0.02165212482213974
Epoch 0 batch idx 31971, loss 0.01927936263382435
Epoch 0 batch idx 31972, loss 0.019948910921812057
Epoch 0 batch idx 31973, loss 0.019946515560150146
Epoch 0 batch idx 31974, loss 0.0212391447275877
Epoch 0 batch idx 31975, loss 0.020348766818642616
Epoch 0 batch idx 31976, loss 0.017823150381445885
Epoch 0 batch idx 31977, loss 0.017073137685656548
Epoch 0 batch idx 31978, loss 0.022851483896374702
Epoch 0 batch idx 31979, loss 0.01891130

In [19]:
criterion= nn.MSELoss()
epochs = 1
for epoch in range(epochs):
    for batch_idx,(data,target) in enumerate (test_loader):
        
       
        output = net (data.float())
        loss = criterion(output.float(),target.float())
        
        accuracy = torch.sum(loss / (len(data) * 1.0))
    print('Epoch {} batch idx {}, Accuaracy {}'.format(epoch,batch_idx,accuracy))

Epoch 0 batch idx 8006, loss 9.758566739037633e-05
